In [ ]:
import pandas as pd

В предыдущем ноутбуке мы исследовали русскоязычный датасет новостей <br>
сейчас мы попробуем провести весь пайплайн классификации для датасета на английском <br>
при этом, мы можем отметить, что домен - остается таким - же классификация сентимента новостей

# Загрузим данные и проведем EDA

In [ ]:
Датасет англоязычных новостей

In [ ]:
# загрузим датасет через sklrn интерфейс

In [ ]:
from sklearn.datasets import fetch_20newsgroups

#загрузим обучающую выборку

newsgroups_train = fetch_20newsgroups(subset='train')

In [ ]:
newsgroups_train.keys()

In [ ]:
newsgroups_train.filenames

In [ ]:
newsgroups_train.target.shape

In [ ]:
11314 записей в таргете

In [ ]:
pd.Series(newsgroups_train.target).value_counts(1).head()

In [ ]:
import seaborn as sns

sns.displot(newsgroups_train.target, bins=20)

In [ ]:
мы видим, что распределение достаточно равномерное - лишь 3 класса имеют меньше наблюдений

In [ ]:
Посмотрим на пример данных:


In [ ]:
print(newsgroups_train.data[5000])

In [ ]:
#соответствующая тематика:
newsgroups_train.target_names[newsgroups_train.target[5000]]

In [ ]:
20 классов
18к записей

In [ ]:
X = newsgroups_train.data
y = newsgroups_train.target

X = pd.DataFrame(X)
X.columns = ['text']

In [ ]:
X.head()

Мы видим следующие поля text, id и sentiment (метка класса) <br>
при этом, для теста нет метки класса - это сделано для того, чтобы узнавать оценку на кэгл - лидерборде

### Посмотрим на распределение таргета

In [ ]:
pd.Series(y).value_counts(1).head() #равномерное распределение таргета

In [ ]:
Мы видим, что почти половина новостей - нейтральные, на позитивные приходится треть и оставшиеся 17% на негативные

## Оценим распределение длительности предложений (текстов)

In [ ]:
# токенизируем по пробелам

sns.displot(X.text.apply(lambda x: len(x.split())), log_scale=True)

In [ ]:
Большинство текстов имеет от 50 до 1000 символов
Распределение похоже на лог-нормальное

In [ ]:
# токенизируем по пробелам

X.text.apply(lambda x: len(x.split())).describe()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
1) Привести в lower-case
2) Убрать знаки пунктуации
3) Токенизация по словам
4) Стемминг/леммитизация
5) ..

## Punctuation Removal: 


In [ ]:
#library that contains punctuation
import string
string.punctuation

In [ ]:
уберем пунктуацию (знаки препинания)

## Stop word removal

In [ ]:
#Stop words present in the library

from nltk.corpus import stopwords
from pymystem3 import Mystem



In [ ]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

In [ ]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
stopwords = stopwords.words("english")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in stopwords\
              and token != " " \
              and token.strip() not in string.punctuation]
    
    text = " ".join(tokens)
    
    return text


In [ ]:
будем использовать Snowball Stemmer

In [ ]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language='english')

In [ ]:
preprocess_text("I was singing in the rain, just flowers growing in the garden")



In [ ]:
preprocess_text("I was singing in the rain, just flowers growing in the garden")



In [ ]:
preprocess_text_stem("I was singing in the rain, just flowers growing in the garden")



In [ ]:
# Заменим лемматизатор в пайплайне на стеммы
#проблема с очисткой от пунктуации

#Preprocess function
def preprocess_text_stem(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [snowball.stem(x) for x in text.lower().split()] 
    tokens = [token for token in tokens if token not in stopwords\
              and token != " " \
              and token.strip() not in string.punctuation]
    
    text = " ".join(tokens)
    
    return text


In [ ]:
print(X.loc[8261].text)

In [ ]:
print(preprocess_text(X.loc[8261].text))

In [ ]:
print(preprocess_text_stem(X.loc[8261].text))

In [ ]:
Скорее всего, для английского языка больше подходит стемминг, а не лемматизация
можем это проверить на валидационной выборке

## Применим все шаги и получим токены:

обратим внимание, что TfidfVectorizer принимает на вход строку - а не лист, те токенизацию производит самостоятельно

In [ ]:
X['tokenized'] = X.text.apply(lambda x: preprocess_text_stem(x))

In [ ]:
X.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
Векторизируем текст 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
X = X.tokenized
y = y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
vectoriser = TfidfVectorizer()

X_train = vectoriser.fit_transform(X_train)
# Convert sparse matrix to dataframe
#X_train = pd.DataFrame.sparse.from_spmatrix(X_train)

# Transform to feature matrix
X_test = vectoriser.transform(X_test)
# Convert sparse matrix to dataframe
#X_test = pd.DataFrame.sparse.from_spmatrix(X_test)
# Add column names to make it more readible


In [ ]:
X_train.shape, X_test.shape

In [ ]:
107479  - очень сильно раздулась размерность!

In [ ]:
Размерность после степинга  - 96752 - чуть меньше в сравнении с лемматизацией

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
Решение:
    увеличить кол-во итераций +
    стандартизовать/нормализовать данные
    
рассмотрим все варианты

In [ ]:
lg = LogisticRegression(max_iter=10000)

In [ ]:
lg.fit(X_train, y_train) # сошлось

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# лемматизация

print(classification_report(y_test, lg.predict(X_test))) #качество на тесте

In [ ]:
# стемминг

print(classification_report(y_test, lg.predict(X_test))) #качество на тесте

In [ ]:
кажется, что стемминг работает чуть лучше для английского текста, что и ожидаемо
сравним с результатами на тесте

Как мы видим, обычный лог рег довольно хорошо справился с проблемой

Сравним данное решение с 
    - Стемминг вместо лемматизации
    - Возьмем готовые эмбеддинги из датасета

In [ ]:
Измерим качество на второй части датасета - 

In [ ]:
newsgroups_test = fetch_20newsgroups(subset='test')

In [ ]:
print(newsgroups_test.data[10])

In [ ]:
newsgroups_test.target, newsgroups_test.target.shape, len(newsgroups_test.data)

In [ ]:
X_holdout = newsgroups_test.data
y_hold = newsgroups_test.target

In [ ]:
X_holdout = pd.DataFrame(X_holdout)

X_holdout.columns = ['text']

X_holdout.head()

In [ ]:
X_holdout.shape

In [ ]:
X_holdout['tokenized'] = X_holdout.text.apply(lambda x: preprocess_text_stem(x))

In [ ]:

valid = vectoriser.transform(X_holdout['tokenized'])
# Convert sparse matrix to dataframe
#X_train = pd.DataFrame.sparse.from_spmatrix(X_train)




In [ ]:
pred = lg.predict(valid)

In [ ]:
pred.shape

In [ ]:
print(classification_report(y_hold, pred)) #качество на тесте

In [ ]:
print(classification_report(y_hold, pred)) #качество на тесте

In [ ]:
На отложенной выборке уже нельзя однозначно сказать, что лучше - лемматизация или стемминг

In [ ]:
Возможно тут все зависит от более тонкой и проработанной настройки пайплайна

In [ ]:
Видим, что качество на отложенной выборке существенно просело, но остается на вполне достойном уровне

In [ ]:
Сравним результат с стеммами вместо лематизации